# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
path = "../Output/city_data.csv"
csv = pd.read_csv(path)
csv_path.head()


,Unnamed: 0,City,Lat,Lng,Max Temp.,Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,0,Rikitea,-23.12,-134.97,296.83,81,100,9.08,PF,1603082476
1,1,Posse,-14.09,-46.37,295.88,70,96,2.42,BR,1603082476
2,2,Vardø,70.37,31.11,271.15,79,20,5.10,NO,1603082476
3,3,Port Keats,-14.25,129.55,304.66,57,58,4.70,AU,1603082476
4,4,Hilo,19.73,-155.09,298.15,78,90,2.10,US,1603082477


In [ ]:
vacay_max_temp = csv_path["Max Temp."] 

(city_df[city_df.Lat <= 0])
heatmap_df

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
